In [1]:
# Import buit-in modules

import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.optimize import minimize

%matplotlib notebook
plt.rcParams.update({'font.size': 17})

In [2]:
# Import my own modules
import sys
sys.path.insert(0, '../utils')
from mycvx import *
from myplots import *

## 6.1

In [11]:
Q1 = sp.MatrixSymbol('Q1', 4, 4)
Q2 = sp.MatrixSymbol('Q2', 4, 4)
Q3 = sp.MatrixSymbol('Q3', 4, 4)
Q4 = sp.MatrixSymbol('Q4', 4, 4)
Q = sp.BlockMatrix([[Q1, Q2, Q3, Q4],
                   [Q2, Q1, Q2, Q3],
                   [Q3, Q2, Q1, Q2],
                   [Q4, Q3, Q2, Q1]])

In [12]:
Q1_num = sp.Matrix([[12, 8, 7, 6],
                    [8, 12, 8, 7],
                    [7, 8, 12, 8],
                    [6, 7, 8, 12]])

Q2_num = sp.Matrix([[3, 2, 1, 0],
                    [2, 3, 2, 1],
                    [1, 2, 3, 2],
                    [0, 1, 2, 3]])

Q3_num = sp.Matrix([[2, 1, 0, 0],
                    [1, 2, 1, 0],
                    [0, 1, 2, 1],
                    [0, 0, 1, 2]])

Q4_num = sp.eye(4)

Q = Q.as_explicit()
Q = Q.evalf(subs={Qk: Qk_num for Qk, Qk_num in zip([Q1,Q2,Q3,Q4],[Q1_num,Q2_num,Q3_num,Q4_num])})

In [13]:
b = -sp.Matrix([1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0])

In [14]:
def loss1(x):
    return (x.T*Q*x)/2 + b.T*x

In [15]:
x0 = np.zeros((16,1))
xHist = {}
fxHist = {}
nbEvalList = {}

print('\nUsing \'Conjugate gradient\' - algo 6.2')
xHist, fxHist, nbEvalList = generalDescentMethod(x0, loss1, conjugateMethod, \
                                                eps=1e-6)
print('\nx min', xHist[-1])
print('\nf(x min)', fxHist[-1])
print('\nnb evals', nbEvalList)


Using 'Conjugate gradient' - algo 6.2

x min [ 0.03423704  0.02423337  0.02423337  0.03423704 -0.00143237 -0.02088798
 -0.02088798 -0.00143237  0.03321913  0.02738606  0.02738606  0.03321913
 -0.00663057 -0.00486331 -0.00486331 -0.00663057]

f(x min) [[-0.1190756]]

nb evals {'nbFunEval': 9, 'nbGradEval': 9, 'nbHessEval': 9}


## 6.2

In [3]:
def loss2(x):
    return 100*(x[1] - x[0]**2)**2 + (1 - x[0])**2

res = minimize(loss2, (0,0), method='nelder-mead', options={'xtol': 1e-6, 'disp': False})
loss1Min = res.x
print('xmin=',res.x)
print('f(xmin)=',res.fun)

xmin= [ 1.00000007  1.00000011]
f(xmin)= 3.43174783409e-14


In [18]:
x0s = np.array(((-2.0, 2.0), (2.0, -2.0), (-2.0, -2.0)))
alpha=0.15
beta=0.5

print('Using Fletcher-Reeves algorithm w/ line search params alpha={:.3f}, beta={:.3f}'.format(alpha,beta))
for x0 in x0s:
    xHist = {}
    fxHist = {}
    nbEvalList = {}
    print()
    print(x0)
    xHist, fxHist, nbEvalList = generalDescentMethod(x0, loss2, conjugateMethod, eps=1e-6,
                                                     lineSearchMethod=backtrackingLineSearch,
                                                     alpha=alpha,beta=beta)
    print('x min', xHist[-1])
    print('f(x min)', fxHist[-1])
    print('nb evals',nbEvalList)

Using Fletcher-Reeves algorithm w/ line search params alpha=0.150, beta=0.500

[-2.  2.]
x min [ 1.00023612  1.0004733 ]
f(x min) 5.58532165735e-08
nb evals {'nbFunEval': 4035, 'nbGradEval': 407}

[ 2. -2.]
x min [ 1.00020202  1.00040557]
f(x min) 4.10353033189e-08
nb evals {'nbFunEval': 212, 'nbGradEval': 22}

[-2. -2.]
x min [ 1.00020931  1.00041952]
f(x min) 4.38841004075e-08
nb evals {'nbFunEval': 518, 'nbGradEval': 55}


## 6.3

In [7]:
def loss3(x):
    return 5*x[0]**2 - 9*x[0]*x[1] + 4.075*x[1]**2 + x[0]

res = minimize(loss3, (0,0), method='nelder-mead', options={'xtol': 1e-6, 'disp': False})
loss1Min = res.x
print('xmin=',res.x)
print('f(xmin)=',res.fun)

xmin= [-16.29999878 -17.99999861]
f(xmin)= -8.15


In [22]:
x0 = np.array((1, 1))
xHistCM = {}
fxHistCM = {}
nbEvalCMList = {}

print('Using conjugate gradient algorithm w/o line search')

xHistCM, fxHistCM, nbEvalCMList = generalDescentMethod(x0, loss3, conjugateMethod, eps=3e-7)
print('x min', xHistCM[-1])
print('f(x min)', fxHistCM[-1])
print('nb evals',nbEvalCMList)

Using conjugate gradient algorithm w/o line search
x min [-16.3 -18. ]
f(x min) -8.149999999999565
nb evals {'nbGradEval': 3, 'nbHessEval': 3, 'nbFunEval': 3}


In [49]:
xHistSD = {}
fxHistSD = {}
nbEvalSDList = {}

alpha=0.15
beta=0.5

print('Using Steepest Descent algorithm w/ line search params alpha={:.3f}, beta={:.3f}'.format(alpha,beta))

xHistSD, fxHistSD, nbEvalSDList = generalDescentMethod(x0, loss3, steepestDescent, eps=3e-7,
                                                 lineSearchMethod=noLineSearch)
#                                                  alpha=alpha,beta=beta)
print('x min', xHistSD[-1])
print('f(x min)', fxHistSD[-1])
print('nb evals',nbEvalSDList)

Using Steepest Descent algorithm w/ line search params alpha=0.150, beta=0.500
x min [-16.29989582 -17.99988454]
f(x min) -8.14999999967
nb evals {'nbFuncEval': 3607, 'nbGradEval': 1803}


Results on the cost function `loss3` after the first iterations for both algorithm

In [53]:
for i in range(1,3):
    print('\nResults after', i, 'iter')
    print('\nConjugate gradient algorithm:')
    print('x1 =',xHistCM[i])
    print('f(x1) =',fxHistCM[i])

    print('\nGradient descent algorithm:')
    print('x1 =',xHistSD[i])
    print('f(x1) =',fxHistSD[i])


Results after 1 iter

Conjugate gradient algorithm:
x1 = [ 0.87651718  1.0524802 ]
f(x1) = 0.9292130993762135

Gradient descent algorithm:
x1 = [ 0.87651718  1.0524802 ]
f(x1) = 0.929213099376

Results after 2 iter

Conjugate gradient algorithm:
x1 = [-16.3 -18. ]
f(x1) = -8.149999999999597

Gradient descent algorithm:
x1 = [ 0.72660018  0.6997343 ]
f(x1) = 0.785730135922


## 7.7

In [15]:
x0s = np.array(((-2.0, 2.0), (2.0, -2.0), (-2.0, -2.0)))
xHist = {}
fxHist = {}
nbEvalList = {}

alpha = 0.4
beta = 0.9

for x0 in x0s:
    print('Using Quasi-Newton algorithm w/ line search params alpha={:.3f}, beta={:.3f}'.format(alpha,beta))

    print()
    print(x0)
    xHist, fxHist, nbEvalList = generalDescentMethod(x0, loss2, quasiNewton, eps=3e-7,
                                                           lineSearchMethod=backtrackingLineSearch,
                                                           alpha=alpha, beta=beta,
                                                           method=DFP)
    print('x min', xHist[-1])
    print('f(x min)', fxHist[-1])
    print('nb evals',nbEvalList)

Using Quasi-Newton algorithm w/ line search params alpha=0.400, beta=0.900

[-2.  2.]
x min [ 1.00000005  1.00000011]
f(x min) 4.91503231787e-15
nb evals {'nbGradEval': 44, 'nbFunEval': 203}
Using Quasi-Newton algorithm w/ line search params alpha=0.400, beta=0.900

[ 2. -2.]
x min [ 0.91766924  0.85337779]
f(x min) 0.0194592640027
nb evals {'nbGradEval': 20, 'nbFunEval': 440}
Using Quasi-Newton algorithm w/ line search params alpha=0.400, beta=0.900

[-2. -2.]
x min [-0.58620359  0.40056381]
f(x min) 2.84013482007
nb evals {'nbGradEval': 36, 'nbFunEval': 510}


## 7.8

In [8]:
x0 = np.array((0,0))
xHist = {}
fxHist = {}
nbEvalList = {}

alpha=0.1
beta=0.3

print('Using Quasi-Newton algorithm w/ line search params alpha={:.3f}, beta={:.3f}'.format(alpha,beta))

xHist, fxHist, nbEvalList = generalDescentMethod(x0, loss3, quasiNewton, eps=3e-7,
                                                       lineSearchMethod=backtrackingLineSearch,
                                                       alpha=alpha, beta=beta,
                                                       method=BFGS)
print('x min', xHist[-1])
print('f(x min)', fxHist[-1])
print('nb evals',nbEvalList)

Using Quasi-Newton algorithm w/ line search params alpha=0.100, beta=0.300
x min [-16.3 -18. ]
f(x min) -8.15
nb evals {'nbGradEval': 6, 'nbFunEval': 14}
